In [1]:
from src import checkpoint, data, metrics
from src.device import device
from src.model import SRCNN
from src.utils import UtilSRCNN
from torch import optim
from torch.utils.data import DataLoader


In [2]:
train_data = data.SRCNNData("data/T91/cropped", transform=UtilSRCNN.transforms)
eval_data_set5 = data.SRCNNData("data/test/Set5", transform=UtilSRCNN.transforms)
eval_data_set14 = data.SRCNNData("data/test/Set14", transform=UtilSRCNN.transforms)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=2)
eval_loader_set5 = DataLoader(eval_data_set5, batch_size=1, shuffle=False, num_workers=1)
eval_loader_set14 = DataLoader(eval_data_set14, batch_size=1, shuffle=False, num_workers=1)

In [3]:
srcnn = SRCNN().to(device)

optimizer = optim.SGD(
    [
        {"params": srcnn.block_1.parameters(), "lr": 1e-4},
        {"params": srcnn.block_final.parameters(), "lr": 1e-5},
    ],
    momentum=0.9,
)

In [4]:
model_name = "srcnn-gaussian-0.55"
metric = metrics.MetricSRCNN()


In [5]:
start = 0
end = 2000

In [6]:
for i in range(start, end):
    mean_loss, mean_psnr = UtilSRCNN.train(srcnn, optimizer, train_loader)
    print(f"Epoch: {i} loss: {mean_loss:.5f}, psnr: {mean_psnr:.5f}")
    metric.total_train_loss.append(mean_loss)
    metric.total_train_psnr.append(mean_psnr)

    mean_loss_set5, mean_psnr_set5 = UtilSRCNN.eval(srcnn, eval_loader_set5, metric)
    print(f"  Eval (Set5): loss: {mean_loss_set5:.5f}, psnr: {mean_psnr_set5:.5f}")
    metric.total_eval_loss_set5.append(mean_loss_set5)
    metric.total_eval_psnr_set5.append(mean_psnr_set5)

    mean_loss_set14, mean_psnr_set14 = UtilSRCNN.eval(srcnn, eval_loader_set14, metric)
    print(f"  Eval (Set14): loss: {mean_loss_set14:.5f}, psnr: {mean_psnr_set14:.5f}")
    metric.total_eval_loss_set14.append(mean_loss_set14)
    metric.total_eval_psnr_set14.append(mean_psnr_set14)

    curr_psnr = metric.get_eval_score()
    if curr_psnr > metric.best_psnr:
        print(f"  * New best psnr: {curr_psnr}")
        metric.best_epoch = i
        metric.best_psnr = curr_psnr

        checkpoint.save(
            name=f"{model_name}/best.pt",
            model=srcnn.state_dict(),
            optimizer=optimizer.state_dict(),
            **metric.save_checkpoint(),
        )

    if not (i + 1) % 100 or i == end - 1 or i == start:
        metric.best_epoch = i
        checkpoint.save(
            name=f"{model_name}/{i}.pt",
            model=srcnn.state_dict(),
            optimizer=optimizer.state_dict(),
            **metric.save_checkpoint(),
        )


Epoch: 0 loss: 0.08985, psnr: 11.73610
  Eval (Set5): loss: 0.01841, psnr: 17.60540
  Eval (Set14): loss: 0.02370, psnr: 16.55024
  * New best psnr: 16.827910373085423
Model saved to model/export/srcnn-gaussian-0.55/best.pt
Model saved to model/export/srcnn-gaussian-0.55/0.pt
Epoch: 1 loss: 0.03725, psnr: 14.33446
  Eval (Set5): loss: 0.01726, psnr: 17.92716
  Eval (Set14): loss: 0.02170, psnr: 16.93050
  * New best psnr: 17.19277959120901
Model saved to model/export/srcnn-gaussian-0.55/best.pt
Epoch: 2 loss: 0.03473, psnr: 14.63443
  Eval (Set5): loss: 0.01601, psnr: 18.26870
  Eval (Set14): loss: 0.01994, psnr: 17.29431
  * New best psnr: 17.55072563572934
Model saved to model/export/srcnn-gaussian-0.55/best.pt


KeyboardInterrupt: 